In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

train_df = pd.read_csv('sign_mnist_train.csv')
test_df = pd.read_csv('sign_mnist_test.csv')

2023-04-25 18:40:37.676938: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-25 18:40:37.719874: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-25 18:40:38.940050: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
x_train = train_df.iloc[:,train_df.columns != 'label']
y_train =  train_df.iloc[:,train_df.columns == 'label']
y_train = y_train.values.reshape(-1)

x_test = test_df.iloc[:,test_df.columns != 'label']
y_test = test_df.iloc[:,test_df.columns == 'label']
y_test = y_test.values.reshape(-1)

In [3]:
from PIL import Image

train_data = np.array(x_train).reshape(-1, 28, 28, 1)
train_data = np.repeat(train_data, 3, axis=-1)
train_data= np.array([np.resize(img, (32, 32, 3)) for img in train_data])
test_data = np.array(x_test).reshape(-1, 28, 28, 1)
test_data = np.repeat(test_data, 3, axis=-1)
test_data= np.array([np.resize(img, (32, 32, 3)) for img in test_data])


In [4]:
train_data = train_data / 255.0
test_data = test_data / 255.0

In [5]:
train_labels = to_categorical(y_train, num_classes=26)
test_labels = to_categorical(y_test, num_classes=26)

In [6]:
resnet_model = ResNet50(include_top=False, input_shape=(32, 32, 3), weights='imagenet')
x = Flatten()(resnet_model.output)
x = Dense(1024, activation='relu')(x)
predictions = Dense(26, activation='softmax')(x)
model = Model(inputs=resnet_model.input, outputs=predictions)

2023-04-25 18:40:44.261605: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-04-25 18:40:44.261632: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: gpu0.ada.davidson.edu
2023-04-25 18:40:44.261637: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: gpu0.ada.davidson.edu
2023-04-25 18:40:44.261742: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: 530.30.2
2023-04-25 18:40:44.261758: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:203] kernel reported version is: 530.30.2
2023-04-25 18:40:44.261762: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:309] kernel version seems to match DSO: 530.30.2


In [7]:
opt = Adam(lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

model.fit(train_data, train_labels, batch_size=32, epochs=10, validation_data=(test_data, test_labels))

test_loss, test_acc = model.evaluate(test_data, test_labels)
print('Test accuracy:', test_acc)

Epoch 1/10
858/858 [==============================] - 268s 299ms/step - loss: 0.7865 - accuracy: 0.7848 - val_loss: 2.0297 - val_accuracy: 0.4336
Epoch 2/10
858/858 [==============================] - 255s 297ms/step - loss: 0.2291 - accuracy: 0.9422 - val_loss: 2.2903 - val_accuracy: 0.6566
Epoch 3/10
858/858 [==============================] - 255s 298ms/step - loss: 0.1647 - accuracy: 0.9571 - val_loss: 0.7762 - val_accuracy: 0.8473
Epoch 4/10
858/858 [==============================] - 254s 297ms/step - loss: 0.1193 - accuracy: 0.9699 - val_loss: 5.1962 - val_accuracy: 0.3026
Epoch 5/10
858/858 [==============================] - 254s 296ms/step - loss: 0.2197 - accuracy: 0.9435 - val_loss: 1.9290 - val_accuracy: 0.7171
Epoch 6/10
858/858 [==============================] - 254s 296ms/step - loss: 0.1258 - accuracy: 0.9673 - val_loss: 0.7325 - val_accuracy: 0.8413
Epoch 7/10
858/858 [==============================] - 253s 295ms/step - loss: 0.0678 - accuracy: 0.9831 - val_loss: 0.3773 -